In [1]:
## SPECIFY THESE ##
DATASET_FOLDER = '../datasets/event_20240405_18_06_48_fps1_clip_1_0/'

In [2]:
import pyrealsense2 as rs
import json
import numpy as np
import matplotlib.pyplot as plt
import csv
import cv2
import os
from tqdm import tqdm

In [4]:
# Path Configs
PATH_DEPTH  = DATASET_FOLDER + 'depth/'
PATH_COLOR = DATASET_FOLDER + 'color'
PATH_DET = DATASET_FOLDER + 'object_detection'
PATH_DET_CSV = DATASET_FOLDER + 'object_detection_csv'
PATH_OBJ3D_CSV = DATASET_FOLDER + 'object_3Dcoord_csv'
PATH_WHOLESCENE_CSV = DATASET_FOLDER + 'whole_scene_csv'
os.makedirs(PATH_OBJ3D_CSV, exist_ok=True)
os.makedirs(PATH_WHOLESCENE_CSV, exist_ok=True)

In [8]:
# read camera intrinsics
json_path = os.path.join(DATASET_FOLDER, 'camera_intrinsic.json')
with open(json_path, 'r') as file:
    data = json.load(file)
intrinsics = rs.pyrealsense2.intrinsics()
intrinsics.width = data['width']
intrinsics.height = data['height']
intrinsics.ppx = data['intrinsic_matrix'][6]
intrinsics.ppy = data['intrinsic_matrix'][7]
intrinsics.fx = data['intrinsic_matrix'][0]
intrinsics.fy = data['intrinsic_matrix'][4]
intrinsics.model = rs.pyrealsense2.distortion.brown_conrady
intrinsics.coeffs = [0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
'''get obj centers 3D coords'''
all_det_csv = sorted([f for f in os.listdir(PATH_DET_CSV)])
for det_csv in tqdm(all_det_csv):
    OBJ3D_csv_path = os.path.join(PATH_OBJ3D_CSV, det_csv)
    det_csv_path = os.path.join(PATH_DET_CSV, det_csv)
    with open(OBJ3D_csv_path, 'w', newline='') as obj3d_file:
        with open(det_csv_path,'r' ) as det_file:
            reader = csv.reader(det_file)
            next(reader)
            writer = csv.writer(obj3d_file)
            writer.writerow(["Object Class Label","confidence", "x", "y", "z"])

            for row in reader:
                label, confidence, left, top, right, bottom, center_dist = row
                left, top, right, bottom, center_dist = int(left), int(top), int(right), int(bottom), float(center_dist) * 1000
                x, y, z =  rs.rs2_deproject_pixel_to_point(intrinsics, [(left+right)/2, (top+bottom)/2], center_dist)
                writer.writerow([label, confidence, x, y, z])

In [7]:
'''get whole scene 2.5D'''
all_color_frame = sorted([f for f in os.listdir(PATH_COLOR)])
for color_file in tqdm(all_color_frame):
    color_path = os.path.join(PATH_COLOR, color_file)
    depth_path = os.path.join(PATH_DEPTH, color_file.replace(".jpg", ".npz"))
    whole_scene_path = os.path.join(PATH_WHOLESCENE_CSV, color_file.replace(".jpg", ".csv"))
    with open(whole_scene_path, 'w', newline='') as csvfile:
        color = cv2.imread(color_path)
        depth = np.load(depth_path)["arr_0"]
        writer = csv.writer(csvfile)

        points = []
        for h in range(intrinsics.height):
            for w in range(intrinsics.width):
                if h % 2 == 0 and w % 2 == 0: # downsample
                    point3d = rs.rs2_deproject_pixel_to_point(intrinsics, [w, h], depth[h, w])
                    if point3d[2] > 8000: continue
                    points.append(point3d)
                    rgb = color[h, w, :]
                    
                    writer.writerow(point3d + list(rgb))


  0%|          | 0/9671 [00:00<?, ?it/s]

100%|██████████| 9671/9671 [1:01:26<00:00,  2.62it/s]
